<a href="https://colab.research.google.com/github/shahaman06/CMPE297-ShortStory/blob/main/Colab%20Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# import torch.nn as nn
# import torchvision
import urllib.request

from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.nn import BCELoss
from torch import ones, rand
from torchvision import transforms
from modules import Discriminator, Generator, PaintingDataset
from tqdm import tqdm

In [20]:
# Declaring GLOBAL VARS:
BATCH_SIZE = 5
IMG_SHAPE = 208
CHANNELS = 3
EPOCHS = 5
FEATURES = 16
RANDOM_NOISE_SHAPE = 256

In [23]:
# Dataset setup
urllib.request.urlretrieve("https://drive.google.com/file/d/1JMUy-StZGaO5PkQlpS7TsRvoj4T13bZ7/view?usp=sharing", "painting.zip")

('painting.zip', <http.client.HTTPMessage at 0x7f42a8b101f0>)

In [21]:
ds = PaintingDataset(img_shape = IMG_SHAPE)
ds[1].shape # -> torch.Size([3, 208, 208])
loader = DataLoader(ds, shuffle = True, batch_size = BATCH_SIZE)
batch = next(iter(loader))
batch.shape # -> torch.Size([5,3, 208, 208])

ValueError: ignored

In [ ]:
disc = Discriminator().to(device)
gen = Generator().to(device)

lr = 0.0002
epochs = 2

optimD = Adam(disc.parameters(), lr = lr, betas = (0.5, 0.99))
optimG = Adam(gen.parameters(), lr = lr, betas = (0.5, 0.99))

criterion = BCELoss()

real_label = 1
fake_label = 0

# fixed_noise = randn(batch_size, channels_noise, 1, 1).to(device)

for epoch in tqdm(range(epochs)):
    for data in loader:
        batch_size = data.shape[0]

        ## Training discriminator

        '''First feeding the real images'''
        label = (ones(batch_size)* 0.9).to(device)
        output = disc(data).reshape(-1)
        lossD_real = criterion(output, label)

        '''feeding generated images'''
        label = (ones(batch_size) * 0.1).to(device)
        rand_noise = randn((batch_size, RANDOM_NOISE_SHAPE, 1, 1)).to(device)
        fake_image = gen(rand_noise)
        output = disc(fake_image.detach()).reshape(-1)
        lossD_fake = criterion(output, label)

        '''Back propogating discriminator and updating weights'''
        disc.zero_grad()
        lossD = lossD_real + lossD_fake
        lossD.backward()
        optimD.step()

        ## Training generator

        label = ones(batch_size).to(device)
        output = disc(fake_image).reshape(-1)
        lossG = criterion(output, label)

        '''Backpropogating'''
        gen.zero_grad()
        lossG.backward()
        optimG.step()